# 1. 计算图

通过数值微分计算了神经网络的权重参数的梯度（严格来说，是损失函数关于权重参数的梯度）。数值微分虽然简单，也容易实现，但缺点是计算上比较费时间。

要正确理解误差反向传播法，有两种方法：

- 一种是基于数学式；
- 另一种是基于计算图（computational graph）

# 2. 链式法则

复合函数是由多个函数构成的函数。比如，$z = (x + y)^2$ 是由式（5.1）所示的两个式子构成的。

$$ z= t^2 $$
$$ t = x + y $$

如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

$z$ 关于 $x$ 的导数$ \frac{\partial z}{\partial x}$，可以用 $z$ 关于 $t$ 的导数 $\frac{\partial z}{\partial t}$ 和 $t$ 关于 $x$ 的导数 $\frac{\partial t}{\partial x}$ 的乘积表示。
$$ \frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 2t\cdot 1 = 2(x+y) $$

计算图的反向传播从右到左传播信号。反向传播的计算顺序是，先将节点的输入信号乘以节点的局部导数（偏导数），然后再传递给下一个节点。

![img](image/11.chain1.png)

将求导结果带入上图中得到，
![img](image/11.chain2.png)

# 3. 反向传播

## 3.1 加法层的反向传播

以 $z = x + y$ 为对象，观察它的反向传播。$z = x + y$ 的导数可解析性地计算出来。

$$ \frac{\partial z}{\partial x} = 1$$

$$ \frac{\partial z}{\partial y} = 1$$

反向传播将从上游传过来的导数（本例中是$ \frac{\partial L}{\partial z}$）乘以1，然后传向下游。也就是说，因为加法节点的反向传播只乘以1，所以输入的值会原封不动地流向下一个节点。

![img](image\11.add_node.png)

接下来，我们实现加法节点的加法层，如下所示：

In [2]:
class AddLayer:
    def __init__(self):
        pass

    # 正向传播
    def forward(self, x, y):
        out = x + y
        return out

    # 反向传播
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

加法层不需要特意进行初始化，所以 \_\_init\_\_() 中什么也不运行（pass语句表示“什么也不运行”）。加法层的 forward() 接收 x 和 y 两个参数，将它们相加后输出。backward() 将上游传来的导数（dout）原封不动地传递给下游。

## 3.2 乘法层的反向传播

这里我们考虑 $z = xy$。这个式的导数用下式表示：

$$ \frac{\partial z}{\partial x} = y$$

$$ \frac{\partial z}{\partial y} = x$$

乘法的反向传播会将上游的值乘以正向传播时的输入信号的“翻转值”后传递给下游。翻转值表示一种翻转关系，如下图所示，正向传播时信号是 x 的话，反向传播时则是 y；正向传播时信号是 y 的话，反向传播时则是 x。

![img](image/11.multi_node.png)

In [4]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    # 正向传播
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    # 反向传播
    def backward(self, dout):
        dx = dout * self.y # 翻转x和y
        dy = dout * self.x
        return dx, dy

\_\_init\_\_()中会初始化实例变量 x 和 y ，它们用于保存正向传播时的输入值。forward() 接收 x 和 y 两个参数，将它们相乘后输出。backward()将从上游传来的导数（dout）乘以正向传播的翻转值，然后传给下游。